# urban_theory_GAN

Hanna Ginzburg & Kai Matheson

Code from Mattia Spinelli

https://medium.com/@mattiaspinelli/simple-generative-adversarial-network-gans-with-keras-1fe578e44a87

https://github.com/daymos/simple_keras_GAN

In [0]:
import numpy as np
import os
from IPython.core.debugger import Tracer


#from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
plt.switch_backend('agg')   # allows code to run without a system DISPLAY

from google.colab import files
import zipfile
import shutil


In [0]:
uploaded = files.upload()

In [0]:
zip_ref = zipfile.ZipFile('256x256maps.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
ls

In [0]:
mydata = []
for filename in os.listdir('256x256maps'):
    if filename.endswith(".png"): 
        img = mpimg.imread('256x256maps/' + filename)
        img = 255 - cv2.cvtColor( img, cv2.COLOR_RGB2GRAY )
        img = misc.imresize(img, (100,100))
        img = img.reshape([10000])
        mydata.append(img)
        continue
    else:
        continue

In [0]:
mydata = np.asarray(mydata)

In [0]:
mydata.shape
#Now each row represents an image and each column represents a pixel that takes a grayscale value between 0 and 

In [0]:
plt.imshow(mydata[3].reshape([100,100]))

In [0]:
!pip install -r requirements.txt

In [0]:
os.makedirs('images')

In [0]:
class GAN(object):
    """ Generative Adversarial Network class """
    def __init__(self, width=100, height=100, channels=1):

        self.width = width
        self.height = height
        self.channels = channels

        self.shape = (self.width, self.height, self.channels)

        self.optimizer = Adam(lr=0.0002, beta_1=0.5, decay=8e-8)

        self.G = self.__generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.optimizer)

        self.D = self.__discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator()

        self.stacked_generator_discriminator.compile(loss='binary_crossentropy', optimizer=self.optimizer)


    def __generator(self):
        """ Declare generator """

        model = Sequential()
        model.add(Dense(256, input_shape=(100,)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.width  * self.height * self.channels, activation='tanh'))
        model.add(Reshape((self.width, self.height, self.channels)))

        return model

    def __discriminator(self):
        """ Declare discriminator """

        model = Sequential()
        model.add(Flatten(input_shape=self.shape))
        model.add(Dense((int(self.width * self.height * self.channels)), input_shape=self.shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense((int((self.width * self.height * self.channels)/2))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model

    def __stacked_generator_discriminator(self):

        self.D.trainable = False

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model

    def train(self, X_train, epochs=201, batch = 32, save_interval = 10):

        for cnt in range(epochs):

            ## train discriminator
            random_index = np.random.randint(0, len(X_train) - batch/2)
            legit_images = X_train[int(random_index) : int(random_index + batch//2)].reshape(batch//2, self.width, self.height, self.channels)

            gen_noise = np.random.normal(0, 1, (batch//2, 100))
            syntetic_images = self.G.predict(gen_noise)

            x_combined_batch = np.concatenate((legit_images, syntetic_images))
            y_combined_batch = np.concatenate((np.ones((batch/2, 1)), np.zeros((batch/2, 1))))

            d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)


            # train generator

            noise = np.random.normal(0, 1, (batch, 100))
            y_mislabled = np.ones((batch, 1))

            g_loss = self.stacked_generator_discriminator.train_on_batch(noise, y_mislabled)

            print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

            if cnt % save_interval == 0:
                self.plot_images(save2file=True, step=cnt)


    def plot_images(self, save2file=False, samples=16, step=0):
        ''' Plot and generated images '''
        filename = "images/maps_%d.png" % step
        noise = np.random.normal(0, 1, (samples, 100))

        images = self.G.predict(noise)

        plt.figure(figsize=(10, 10))
        
        for i in range(images.shape[0]):
          image = images[i,:,:,:]
          image = np.reshape(image, [self.height, self.width])
          plt.imshow(image)
        
        #for i in range(images.shape[0]):
            #plt.subplot(4, 4, i+1)
            #plt.subplot(1, 1, i+1)
            #image = images[i, :, :, :]
            #image = np.reshape(image, [self.height, self.width])
            #plt.imshow(image, cmap='gray')
            #plt.imshow(image)
            #plt.axis('off')
        plt.tight_layout()

        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()


if __name__ == '__main__':
    # (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (mydata.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)


    gan = GAN()
    gan.train(X_train)

In [0]:
cd images

In [0]:
ls

In [0]:
imgplot = plt.imshow(mpimg.imread('maps_150.png'))

In [0]:
cd ~

In [0]:
shutil.make_archive('simple_gan_output', 'zip', 'images')

In [0]:
files.download('simple_gan_output.zip')